In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import cv2
import os
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import pi, cos, sin # used for HoF
from scipy.ndimage import uniform_filter # used for hoF
from numpy.lib.scimath import sqrt # used for hoF
from numpy import arctan2 # used for hoF

from sklearn import svm

import math
import numpy as np
import datetime as dt
from collections import deque
import pandas as pd
 
from sklearn.model_selection import train_test_split
 
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, MaxPooling2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint,CSVLogger
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
import tensorflow as tf
# Check GPU coinfiguration in Colab
print("Tensorflow version: ", tf.__version__)
print(tf.test.gpu_device_name())
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

Tensorflow version:  2.12.0
/device:GPU:0


In [ ]:
import os
os.chdir('/content/drive/MyDrive/MTech AI NUS/PM ITSS/')
!ls
directoryName = 'Dataset_flight'

'2D_CNN Model -Diving Dataset.ipynb'
'C3D Model - Diving Dataset.ipynb'
 Dataset
 Dataset_flight
 model_2D1.csv
 model_2D1.h5
 model_2D.csv
 model_2D.h5
 model_c3d.csv
 model_c3d.h5
 model_flightClassify_2D_1.csv
 model_flightClassify_2D_1.h5
 model_flightClassify_2D_2.csv
 model_flightClassify_2D_2.h5
 model_flightClassify_2D_3.csv
 model_flightClassify_2D_3.h5
 model_flightClassify_2D_4.csv
 model_flightClassify_2D_4.h5
 model_flightClassify_2D_5.csv
 model_flightClassify_2D_5.h5
 model_flightClassify_2D.csv
 model_flightClassify_2D.h5
 Temporal_segmentation_final_pipeline.ipynb
 VIDEO


In [ ]:
dive_action_labels  = sorted(os.listdir(directoryName))
print("Dive labels: ", dive_action_labels)

Dive labels:  ['Somersault', 'Twist']


In [ ]:
image_height, image_width = 64, 64

In [ ]:
def create_model(optim):
 
    model = Sequential()
 
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu', input_shape = (image_height, image_width, 3)))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(256, activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dense(2, activation = 'softmax'))
    model.compile(loss=categorical_crossentropy, optimizer=optim, metrics=['accuracy'])
    # Printing the models summary
    model.summary()
 
    return model

In [ ]:
def predict(video_file_path, output_file_path, window_size,model_CNN):
    
    predicted_label_list = []
    predicted_labels_probabilities_deque = deque(maxlen = window_size)
 
    video_reader = cv2.VideoCapture(video_file_path)
    frame_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    print(frame_count)
 
    # Getting the width and height of the video 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
    # Writing the Overlayed Video Files Using the VideoWriter Object
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 30, (original_video_width, original_video_height))
 
    while True: 
 
        # Reading The Frame
        status, frame = video_reader.read() 
 
        if not status:
            break
 
        # Resize the Frame to fixed Dimensions
        resized_frame = cv2.resize(frame, (image_height, image_width))
         
        # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1
        normalized_frame = resized_frame / 255
 
        # Passing the Image Normalized Frame to the model and receiving Predicted Probabilities.
        predicted_labels_probabilities = model_CNN.predict(np.expand_dims(normalized_frame, axis = 0))[0]
 
        # Appending predicted label probabilities to the deque object
        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)
 
        # Assuring that the Deque is completely filled before starting the averaging process
        if len(predicted_labels_probabilities_deque) == window_size:
 
            # Converting Predicted Labels Probabilities Deque into Numpy array
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)
            print('predicted_labels_probabilities_np : ',predicted_labels_probabilities_np)
 
            # Calculating Average of Predicted Labels Probabilities Column Wise 
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis = 0)
            print('predicted_labels_probabilities_averaged',predicted_labels_probabilities_averaged)
 
            # Converting the predicted probabilities into labels by returning the index of the maximum value.
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)
            predicted_label_list.append(predicted_label)
 
            # Accessing The Class Name using predicted label.
            predicted_class_name = dive_action_labels[predicted_label]
           
            # Overlaying Class Name Text Ontop of the Frame
            cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
 
        # Writing The Frame
        video_writer.write(frame)
 
     
    # Closing the VideoCapture and VideoWriter objects and releasing all resources held by them. 
    video_reader.release()
    video_writer.release()
    return predicted_label_list

In [ ]:
def get_video_frames(output_video_file_path,predicted_label,output_videoPath):
    video_reader = cv2.VideoCapture(output_video_file_path)
 
    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
 
    indices_twist = [i for i, x in enumerate(predicted_label) if x == 1]
    indices_somersault = [i for i, x in enumerate(predicted_label) if x == 0]
    print(indices_twist)
    print(indices_somersault)
    class_list = [indices_somersault,indices_twist]
    
         
    for i, j in enumerate(class_list):
        video_writer = cv2.VideoWriter(output_videoPath+'_'+str(i)+'.avi', cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 30, (original_video_width, original_video_height) ) 
        for indices in j:
            video_reader.set(cv2.CAP_PROP_POS_FRAMES,indices)
            #print('Position:', int(video_reader.get(cv2.CAP_PROP_POS_FRAMES)))
            _, frame = video_reader.read()
            video_writer.write(frame)
        #video_reader.release()
        video_writer.release()

In [ ]:
def create_temporal_segmentation_flight(video_filePath,output_video_file_path):
    #image_folder = 'VIDEO/Video_Images/04'
    #video_name = 'VIDEO/video_05.avi'

    optim = tf.keras.optimizers.Adam(0.0001)
    model_CNN = create_model(optim)
    model_CNN.load_weights("model_flightClassify_2D_3.h5")
    window_size = 1
    predicted_label = predict(video_filePath, output_video_file_path, window_size,model_CNN)
    output_class_folder = video_filePath[:-4]+'_folder'
    get_video_frames(video_filePath,predicted_label,output_class_folder)    

    return predicted_label

In [ ]:
predicted_label = create_temporal_segmentation_flight('VIDEO/Video_Flight_Images/04/video_13_folder_1.avi','VIDEO/videoFlightClassification_04.avi')
print(predicted_label)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 62, 62, 64)        1792      
                                                                 
 conv2d_7 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 batch_normalization_4 (Batc  (None, 60, 60, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 30, 30, 64)       0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 28, 28, 64)        36928     
                                                                 
 global_average_pooling2d_2   (None, 64)              